In [ ]:
# default_exp utils

# Utils
> Utilities

In [ ]:
# exporti
from jax.lax import conv
from functools import wraps

In [ ]:
# export
@wraps(conv)
def conv2d(lhs, rhs, window_strides=(1,1), padding="SAME", **kwargs):
    return conv(lhs[None, None, :, :], rhs[None, None, :, :], window_strides, padding, **kwargs)[0, 0, :, :]

In [ ]:
# export
@wraps(conv)
def batch_conv2d(lhs, rhs, window_strides=(1,1), padding="SAME", **kwargs):
    return conv(lhs[:, None, :, :], rhs[:, None, :, :], window_strides, padding, **kwargs)[:, 0, :, :]

In [ ]:
# export
def dilute(touches, brush):
    return conv2d(
        lhs=touches,
        rhs=brush,
        window_strides=(1, 1),
        padding="SAME",
        preferred_element_type=bool,
    )